In [1]:
# Import necessary libraries

# ! pip install fredapi
# ! pip install urllib

from fredapi import Fred  # fredapi
import pandas as pd       # general use
import urllib             # fredapi

# FRED API

In [2]:
# Store FRED API Key
fred = Fred(api_key='ee9c4161f6205d40418acb6d19990502')

### Real Sales

In [3]:
df = fred.get_series_all_releases('RSXFS')

In [4]:
df_rs = df[["date", "value"]]  # Store only date and gdp value columns in new df
df_rs = df_rs.groupby('date').nth(-1)  # Retain last record for each date
df_rs.rename(columns={"value": "real_sales"}, inplace = True)  # Rename values column

### Capacity Utilization: Total Index

In [5]:
# Pull Capacity Utilization: Total Index Data
df = fred.get_series_all_releases('TCU')

In [6]:
df_tcu = df[["date", "value"]]  # Store only date and gdp value columns in new df
df_tcu = df_tcu.groupby('date').nth(-1)  # Retain last record for each date
df_tcu.rename(columns={"value": "tcu"}, inplace = True)  # Rename values column

### Industrial Production

In [7]:
# Pull INDPRO
df = fred.get_series_all_releases('INDPRO')

In [8]:
df_ip = df[["date", "value"]]  # Store only date and unemp value columns in new df
df_ip = df_ip.groupby('date').nth(-1)  # Retain last record for each date
df_ip.rename(columns={"value": "indpro"}, inplace = True)  # Rename values column

### SAHM

In [9]:
# Pull SAHM
df = fred.get_series_all_releases('SAHMREALTIME')

In [10]:
df_s = df[["date", "value"]]  # Store only date and unemp value columns in new df
df_s = df_s.groupby('date').nth(-1)  # Retain last record for each date
df_s.rename(columns={"value": "SAHM"}, inplace = True)  # Rename values column

## Combine FRED DFs

In [11]:
df_hni = df_tcu.merge(df_ip, left_index=True, right_index=True)

dfs = [df_rs, df_s]

for i in dfs:
   df_hni = df_hni.merge(i, left_index=True, right_index=True)

# Import Provided Files

### Sales

In [12]:
# Import sales report from file
df_sales = pd.read_csv("data/Sales Data.csv", sep= ",")
df_sales.rename(columns={"Unnamed: 0": "month"}, inplace = True)  # Rename values column

In [13]:
df_sales = pd.Series(df_sales.values.ravel('F'))

df_sales = df_sales.drop([0,1,2,3,4,5,6,7,8,9,10,11])
df_sales = df_sales.dropna()

In [14]:
df_sales = df_sales.reset_index(drop=True)

In [15]:
# Create a date column based on length of sales records
x = len(df_sales)

dt_s = pd.Series(
    pd.date_range("2000-01-01", periods=x, freq="M")
)

dt_s = dt_s - pd.offsets.MonthBegin(1)

In [16]:
# Create furniture industry sales df
df_fis = pd.concat([dt_s, df_sales], axis=1)
df_fis = df_fis.rename(columns={0: "date", 1: "Sales"})
df_fis['date'] = df_fis['date'].astype('datetime64[ns]') 
df_fis = df_fis.set_index('date')  # set index to date col
df_fis.Sales = df_fis.Sales.replace({'\$':''}, regex = True)
df_fis.Sales = df_fis.Sales.replace({'\,':''}, regex = True)
df_fis["Sales"] = pd.to_numeric(df_fis["Sales"]) # Convert value to a numeric column

In [17]:
df_full = df_hni.merge(df_fis, left_index=True, right_index=True)

In [18]:
df_full.reset_index(inplace=True)
df_full.rename(columns={"index": "date"}, inplace = True)  # Rename values column

In [19]:
df_full['lag12'] = df_full['Sales'].shift(12)
df_full = df_full.dropna()
df_full['lag12'] = df_full['lag12'].astype('int')

### Create lag df

In [20]:
df_lag = df_full[['Sales']].shift(-12)

df_lag['date'] = pd.date_range(start='1/1/2003', periods=len(df_full), freq='MS') 
df_lag = df_lag.set_index('date')
df_lag.rename(columns={'Sales': 'lag12'}, inplace=True)
df_lag = df_lag.dropna()

df_lag['lag12'] = df_lag['lag12'].astype('int')

### Percent Change Column

In [21]:
# Create column display monthly sales percent change from prior year value
df_full['p_c'] = ((df_full['Sales'] - df_full['lag12']) / df_full['lag12']) * 100

In [22]:
df2 = df_full.loc[:,['date', 'p_c']]
df_full = df_full.drop(columns=['p_c'])

## Create dataframe for months with known variables for unknown sales

In [23]:
d1 = df_hni.index[-1]   # Store most recent date with complete FRED data
d2 = df_fis.index[-1]   # Store most recent date with sales data

In [24]:
m = round(abs((d1-d2).days)/30)   # Determine periods with FRED data & without sales

In [25]:
df_pred = df_hni.tail(m)   # Store these rows in new df

In [26]:
# Create tmp sales df for lagged sales
df_f2 = df_fis
df_f2['new_date'] = df_fis.index - pd.DateOffset(years=-1)
df_f2.index = df_f2.new_date

# Merge lagged sales with pred rows to provide complete records for modeling
df_pred = df_pred.merge(df_f2, left_index=True, right_index=True)

# Clean df pred
df_pred.rename(columns={"Sales": "lag12"}, inplace = True)
df_pred = df_pred.drop(columns=['new_date'])

# Write to CSV

In [27]:
# DF containing consolidated data

df = df_full
df.to_csv("data/HNI.csv", sep=",", index=False, header= True)

In [28]:
# DF containing the sales percent change data

df = df2
df.to_csv("data/HNI_pc.csv", sep=",", index=True, header=True)

In [29]:
# DF containing records with known input variables, but unknown sales

df = df_pred
df.to_csv("data/HNI_pred.csv", sep=",", index=True, header=True)

In [30]:
# DF containing consolidated FRED data

df_hni = df_hni.dropna()
df = df_hni
df.to_csv("data/HNI_all.csv", sep=",", index=True, header=True)

In [31]:
# DF containing lagged sales information

df = df_lag
df.to_csv("data/HNI_lag.csv", sep=",", index=True, header=True)